<h1>Emotion Detection and Information Retrieval</h1>

<h2>Installing Libraries</h2>

In [15]:
#!pip install duckduckgo-search
#!pip install search-engines

  Created wheel for search-engines: filename=search_engines-1.0.7-py3-none-any.whl size=9589 sha256=a84023b17665dd586393df97c9c342d0a752ef35983382fa47c6ebb63414a0da
  Stored in directory: c:\users\lvallejos\appdata\local\pip\cache\wheels\67\62\a9\9c4addc6e130e3866a6743c13be44ced18b2ea2dd51995e02a
Successfully built search-engines


<h2>Defining Relevant Tokens</h2>

In [55]:
cryptocurrency_name = "bitcoin"
cryptocurrency_symbol = "$BTC"
keywords = f"{cryptocurrency_name} {cryptocurrency_symbol} sentiment"
keywords_for_news = f"{cryptocurrency_name} crypto"

<h2>Retrieve Links from Search Engines</h2>

In [ ]:
# References:
############# DuckDuckGo
# (GoogleSearch) 1. https://medium.com/@nutanbhogendrasharma/how-to-scrape-google-search-engines-in-python-44770b8eab5
# (DuckDuckGo)   2. https://pypi.org/project/duckduckgo-search/
# (DuckDuckGo vs GoogleSearch) 3. https://medium.com/hackernoon/duckduckgo-vs-google-what-you-need-to-know-869368b08c4f
# (DuckDuckGo vs GoogleSearch) 4. https://www.cnet.com/tech/mobile/in-ios-17-apple-adds-ability-to-change-search-engine-in-safari-private-browsing/

############# Serch engines like Bing or Yahoo
#https://pypi.org/project/search-engines/

<h3> Importing Libraries </h3>

In [51]:
from duckduckgo_search import DDGS
from search_engines import google_search, bing_search, yahoo_search, yahoo_news, google_news, bing_news
import requests

In [23]:
MAX_SITES_RESULTS = 100
TIMEOUT_SECONDS = 5

<h3>Functions</h3>

In [24]:
def get_results(search_engine, page_url):
    try:
        response = requests.get(page_url, timeout=TIMEOUT_SECONDS)
        response.raise_for_status()  # Raise an exception for HTTP errors
        html = response.text
        return search_engine.extract_search_results(html, page_url), response.url
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during 'get_results' function execution: {e}")
        return [], None

In [72]:
def fetch_search_results(search_engine, query, max_results):
    search_results = []
    search_url = search_engine.get_search_url(query)

    while len(search_results) < max_results and search_url:
        try:
            next_search_results, search_url = get_results(search_engine, search_url)
            for result in next_search_results:
                # Ensure each result is a dictionary with a "url" key
                if "url" in result:
                    search_results.append(result)
        except Exception as e:
            print(f"An error occurred during 'fetch_search_results' function execution: {e}")
            break

    return search_results[:max_results]


<h3>DuckDuckGo for SearchEngine</h3>

In [31]:
ddgs_results = list(map(lambda r: r["href"], DDGS().text(keywords, max_results=MAX_SITES_RESULTS)))
ddgs_results

['https://alternative.me/crypto/fear-and-greed-index/',
 'https://www.augmento.ai/bitcoin-sentiment/',
 'https://coincodex.com/sentiment/',
 'https://www.lookintobitcoin.com/charts/bitcoin-fear-and-greed-index/',
 'https://www.cryptoeq.io/sentiment-report/sentiment/bitcoin',
 'https://www.bittsanalytics.com/sentiment-index/BTC',
 'https://www.statista.com/statistics/1343514/bitcoin-sentiment/',
 'https://www.bloomberg.com/news/articles/2023-10-03/bitcoin-btc-price-dips-from-six-week-high-amid-bond-rout',
 'https://www.marketbeat.com/cryptocurrencies/bitcoin/',
 'https://www.binance.com/en/price/bitcoin',
 'https://www.dailyfx.com/bitcoin',
 'https://www.fxstreet.com/rates-charts/btcusd',
 'https://cryptonews.com/news/bitcoin-price-prediction-as-btc-smashes-past-27000-resistance-best-time-buy.htm',
 'https://cryptonews.com/news/bitcoin-price-prediction-btc-rises-4-amidst-dovish-fomc-sentiment.htm',
 'https://www.investing.com/crypto/bitcoin/bitcoin-futures-scoreboard',
 'https://www.coi

<h3>Bing for SearchEngine</h3>

In [61]:
bing_results = []
for result in fetch_search_results(bing_news, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        bing_results.append(result["url"])
        
for result in fetch_search_results(bing_search, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        bing_results.append(result["url"])

bing_results

Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracted 0 results from page .
Extracte

[]

<h3>Yahoo for SearchEngine</h3>

In [62]:
yahoo_results = []
for result in fetch_search_results(yahoo_news, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        yahoo_results.append(result["url"])
        
for result in fetch_search_results(yahoo_search, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        yahoo_results.append(result["url"])

yahoo_results

[]

<h3>Google for SearchEngine</h3>

In [60]:
google_results = []
for result in fetch_search_results(google_news, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        google_results.append(result["url"])
        
for result in fetch_search_results(google_search, keywords, MAX_SITES_RESULTS):
    if isinstance(result, dict) and "url" in result:
        google_results.append(result["url"])

google_results

[]

In [33]:
fetch_search_results(yahoo_search, keywords, MAX_SITES_RESULTS)

[[{'url': 'https://coincodex.com/sentiment/',
   'title': "Crypto Sentiment | CoinCodex A Look Back at Bitcoin 's Performance in 2022 Bitcoin Sentiment – Bull & Bear Index – Augmento Crypto Fear & Greed Index - Bitcoin Sentiment - Alternative.me",
   'preview_text': '',
   'page_number': '1'},
  {'url': 'https://alternative.me/crypto/fear-and-greed-index/',
   'title': 'alternative.me › crypto › fear-and-greed-index Crypto Fear & Greed Index - Bitcoin Sentiment - Alternative.me',
   'preview_text': 'Each day, we analyze emotions and sentiments from different sources and crunch them into one simple number: The Fear & Greed Index for Bitcoin and other large cryptocurrencies. Historical Values Now ',
   'page_number': '1'},
  {'url': 'https://www.augmento.ai/bitcoin-sentiment/',
   'title': 'www.augmento.ai › bitcoin-sentime Bitcoin Sentiment – Bull & Bear Index – Augmento',
   'preview_text': 'Bitcoin  sentiment – Bull & Bear Index. The graphs show how bullish/bearish conversations about

In [76]:
yahoo_results = fetch_search_results(yahoo_search, keywords, MAX_SITES_RESULTS)
yahoo_results

KeyboardInterrupt: 

In [74]:
type(yahoo_results) 

dict

In [69]:
yahoo_results[0]

[{'url': 'https://www.msn.com/en-us/money/careers/a-look-back-at-bitcoins-performance-in-2022/vi-AA15P9Zx',
  'title': "A Look Back at Bitcoin 's Performance in 2022 Crypto Sentiment | CoinCodex Bitcoin Sentiment – Bull & Bear Index – Augmento Crypto Fear & Greed Index - Bitcoin Sentiment - Alternative.me",
  'preview_text': '',
  'page_number': '1'},
 {'url': 'https://alternative.me/crypto/fear-and-greed-index/',
  'title': 'alternative.me › crypto › fear-and-greed-index Crypto Fear & Greed Index - Bitcoin Sentiment - Alternative.me',
  'preview_text': 'Each day, we analyze emotions and sentiments from different sources and crunch them into one simple number: The Fear & Greed Index for Bitcoin and other large cryptocurrencies. Historical Values Now ',
  'page_number': '1'},
 {'url': 'https://www.augmento.ai/bitcoin-sentiment/',
  'title': 'www.augmento.ai › bitcoin-sentime Bitcoin Sentiment – Bull & Bear Index – Augmento',
  'preview_text': 'Bitcoin  sentiment – Bull & Bear Index. The

<h2>HTTP Requests for Page Retrieval</h2>

In [ ]:
# References:
# (BeautifulSoup) 1. https://ai.plainenglish.io/mastering-web-scraping-and-sentiment-analysis-with-python-and-machine-learning-255d1d6234c5

<h2>Filter Out Irrelevant Pages</h2>

<h2>Parse HTML Content and Extract Text</h2>

<h2>Sentiment Analysis</h2>

<h2>Storage of Sentiment and Pages</h2>